In [46]:
from typing import List

import math
import numpy as np
import sympy as sp

In [47]:
# 定义一个经过 (1,0,0)，方向为 (1,1,1) 的直线
l1 = sp.Ray((1,0,0), (1,1,1))

# 定义一个经过 (1,1,1), 法线方向为 (1,1,1) 的平面
s1 = sp.Plane((1,1,1), (1,1,1))

# 计算直线与平面的交点
p1 = s1.intersection(l1)
p1

[Point3D(1, 1, 1)]

In [48]:
import utilities

In [49]:
pos0 = utilities.SingleDetector(center_depth=100, slope_angle=sp.pi/12, direction_angle=2*sp.pi/3, detector_angle=sp.pi/3, precision=20)
pos0.detect_eval()
pos0.scan_segment_length_proj_xy

117.58052938602825217

In [50]:
pos1 = utilities.SingleDetector(center_depth=100, slope_angle=sp.pi/6, direction_angle=3*sp.pi/4, detector_angle=sp.pi/3)

In [51]:
center_depth = pos1.center_depth
slope_angle = pos1.slope_angle
direction_angle = pos1.direction_angle
detector_angle = pos1.detector_angle

origin = sp.Matrix([0, 0, 0])
i_hat = sp.Matrix([1, 0, 0]) # x unit, also slope direction's projection onto the x-y plane
j_hat = sp.Matrix([0, 1, 0]) # y unit
k_hat = sp.Matrix([0, 0, 1]) # z unit

loc = sp.Matrix([0, 0, center_depth])
# 坡面法向量
slope_normal_vec = sp.Matrix([sp.sin(slope_angle), 0, sp.cos(slope_angle)])
# 坡面
slope = sp.Plane(origin, normal_vector=slope_normal_vec)

slope

Plane(Point3D(0, 0, 0), (1/2, 0, sqrt(3)/2))

In [52]:
# 船的方向向量 - direction vector
direction_vec = sp.Matrix([sp.cos(direction_angle), sp.sin(direction_angle), 0])
# 船的方向向量在x-y平面上旋转90度，得到船的侧向向量 - side vector
side_vec = sp.Matrix([-sp.sin(direction_angle), sp.cos(direction_angle), 0])

side_vec

Matrix([
[-sqrt(2)/2],
[-sqrt(2)/2],
[         0]])

In [53]:
# 探测器单边检测角度
detector_angle_half = detector_angle/2
detector_angle_half

pi/6

In [54]:
# 探测器检测扇形与x-y平面的两个交点
detect_point_xy_1 = side_vec * sp.tan(detector_angle_half) * center_depth
detect_point_xy_2 = -side_vec * sp.tan(detector_angle_half) * center_depth

detect_point_xy_1
detect_point_xy_2

Matrix([
[50*sqrt(6)/3],
[50*sqrt(6)/3],
[           0]])

In [55]:
# 检测扇形的两个边沿
detect_line_1 = sp.Line(loc, detect_point_xy_1)
detect_line_2 = sp.Line(loc, detect_point_xy_2)

detect_line_1
detect_line_2

Line3D(Point3D(0, 0, 100), Point3D(50*sqrt(6)/3, 50*sqrt(6)/3, 0))

In [56]:
# 检测扇形的两个边沿与坡面的交点
detect_point_1 = slope.intersection(detect_line_1)
detect_point_2 = slope.intersection(detect_line_2)
assert len(detect_point_1) == 1 and len(detect_point_2) == 1
detect_point_1 = detect_point_1[0]
detect_point_2 = detect_point_2[0]

detect_point_1
detect_point_2

Point3D(50*sqrt(6)*(3*sqrt(2)/17 + 18/17)/3, 50*sqrt(6)*(3*sqrt(2)/17 + 18/17)/3, -300*sqrt(2)/17 - 100/17)

In [57]:
# 检测扇形的两个边沿与坡面的交点的连线
detect_scan_line = sp.Segment(detect_point_1, detect_point_2)
detect_scan_line

Segment3D(Point3D(-50*sqrt(6)*(18/17 - 3*sqrt(2)/17)/3, -50*sqrt(6)*(18/17 - 3*sqrt(2)/17)/3, -100/17 + 300*sqrt(2)/17), Point3D(50*sqrt(6)*(3*sqrt(2)/17 + 18/17)/3, 50*sqrt(6)*(3*sqrt(2)/17 + 18/17)/3, -300*sqrt(2)/17 - 100/17))

In [58]:
# detect_scan_line与x-y平面的夹角
detect_scan_angle = sp.Abs(sp.pi/2 - sp.Abs(detect_scan_line.angle_between(sp.Line(origin, k_hat))))

detect_scan_angle

-pi/2 + acos(-sqrt(7)/7)

In [59]:
# 检测扇形的两个边沿与坡面的交点的连线的长度
detect_scan_line_length = detect_scan_line.length

detect_scan_line_length

sqrt(720000/289 + 2*(-50*sqrt(6)*(3*sqrt(2)/17 + 18/17)/3 - 50*sqrt(6)*(18/17 - 3*sqrt(2)/17)/3)**2)

In [60]:
# detect_scan_line_length在x-y平面上的投影长度
detect_scan_line_length_xy = detect_scan_line_length * sp.cos(detect_scan_angle)

detect_scan_line_length_xy

sqrt(42)*sqrt(720000/289 + 2*(-50*sqrt(6)*(3*sqrt(2)/17 + 18/17)/3 - 50*sqrt(6)*(18/17 - 3*sqrt(2)/17)/3)**2)/7

In [61]:
# 测线 与 测线所在竖直平面
measuring_line = sp.Line(origin, direction_vec)
measuring_line_plane = sp.Plane(origin, normal_vector=side_vec)

measuring_line
#measuring_line_plane

Line3D(Point3D(0, 0, 0), Point3D(-sqrt(2)/2, sqrt(2)/2, 0))

In [62]:
# 测线平面与坡面的交线
measuring_slope_line = measuring_line_plane.intersection(slope)
assert len(measuring_slope_line) == 1
measuring_slope_line = measuring_slope_line[0]

measuring_slope_line

Line3D(Point3D(0, 0, 0), Point3D(-sqrt(6)/4, sqrt(6)/4, sqrt(2)/4))

In [65]:
# measuring_slope_line与x-y平面的夹角
measuring_slope_line_angle_xy = (sp.Abs(sp.pi/2 - sp.Abs(measuring_slope_line.angle_between(sp.Line(origin, k_hat)))))
measuring_slope_line_angle_xy

-acos(sqrt(7)/7) + pi/2

In [17]:
pos1.detect_eval()
pos1.scan_segment_length_proj_xy

129.90381056766579701